# Installing libraries

In [1]:
!pip install -q -U langchain
!pip install -q -U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.8 MB/s eta 0:00:00


In [2]:
!pip install --upgrade --quiet google-genai

In [3]:
!pip install gTTS --quiet

In [4]:
!pip install playsound --quiet

  Preparing metadata (setup.py) ... done


# Importing tools

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import Markdown,display
import os

In [6]:
from google.colab import userdata
GEMINI_API_KEY: str = userdata.get('GOOGLE_API_KEY_1')
if(GEMINI_API_KEY):
  print("API Key found")
else:
  print("API Key not found")

API Key found


In [7]:
from google import genai
from google.genai import Client

client: Client = genai.Client(
    api_key=GEMINI_API_KEY,
)

model: str = "gemini-2.0-flash-exp"

# Generating video
###I asked LLM to generate the video by giving prompt as:
Create a 30 second video. In the start 15 seconds show a scene of dark night turning into a bright day time, but only show the sky view, with the sound of cool winds blowing. Then in next 15 seconds show a scene of sea waves going back and forth, with the sound of sea waves.

In [15]:
import gdown
import os

generated_video_url = "https://docs.google.com/uc?id=1735808007210"
generated_video_path = "/content/1735808007210.mp4"

if not os.path.exists(generated_video_path):
    try:
        gdown.download(generated_video_url, generated_video_path, quiet=True)
        print(f"Video downloaded successfully: {generated_video_path}")
    except Exception as e:
        print(f"Error downloading video: {e}")
else:
    print(f"Video already exists at: {generated_video_path}")

Video already exists at: /content/1735808007210.mp4


In [19]:
import time
client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY_1"))
ai_generated_video = "/content/1735808007210.mp4"

In [20]:
import time

def upload_video(ai_generated_video):
        try:
            generated_video = client.files.upload(path=ai_generated_video)

            while generated_video.state == "PROCESSING":
                print('Waiting for video to be processed.')
                time.sleep(15)
                generated_video = client.files.get(name=generated_video.name or "")

            if generated_video.state == "FAILED":
                raise ValueError(f"Video processing failed: {generated_video.state}")
            print(f'Video processing complete: {generated_video.uri}')
            return generated_video
        except Exception as e:
            print(f"An error occurred during video upload: {e}")
            return None

my_video = upload_video(ai_generated_video)

Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/tkn36q7oggo6


In [41]:
from google.genai.types import Content, Part
from IPython.display import Markdown,display
def generated_video_with_langchain(ai_generated_video):
  prompt_template = """
    For each scene in this video,
    generate captions that describe the scene along with any spoken text placed in quotation marks.
    Place each caption into an object with the timecode of the caption in the video.
           """

  response = client.models.generate_content(
    model="gemini-2.0-flash-exp",
  contents=[
      Content(
          role="user",
          parts=[
              Part.from_uri(
                  file_uri=ai_generated_video.uri or "",
                  mime_type=ai_generated_video.mime_type or ""),
            ]),
    Content(
        role="user",
        parts=[Part(text=prompt_template)]
        )
    ]
  )
  print("Video Analysis:")
  display(Markdown(response.text))
print(generated_video_with_langchain(my_video))

Video Analysis:


```json
[
    {
        "timecode": "00:00",
        "caption": "A view of a dark sky full of stars with mountains silhouetted at the bottom. The milky way is visible in the middle of the sky. There is a dark pink object at the top of the frame."
    },
    {
       "timecode": "00:07",
        "caption": "The scene changes to a sky with blue and gray clouds. The sun is visible in the lower right corner."
    },
        {
        "timecode":"00:14",
        "caption": "The scene changes again to a dark ocean with dark gray clouds in the sky. The waves are crashing onto the beach. There are two small concrete jetties visible on the beach."
     }
]
```

None
